# Training

#### Input
X and y actually in %restore -r data_label

#### Output 
1. Reusable trained model at model_dir which is 'model/linear' or 'model/dnn'.


### Read data and label

In [1]:
import tensorflow as tf
import peforth

In [2]:
%store -r data_label
X = data_label[0]  # features pandas data-frame
y = data_label[1]  # labels pandas series

In [3]:
# 核對一下 shapes 
%f __main__ :> X.shape . cr
%f __main__ :> y.shape . cr

(4483, 751)
(4483,)


有甚麼 tf.estimator.inputs.??? 適合 751 個 numpy.float64?

In [6]:
tf.__version__

'1.8.0'

In [9]:
help(tf.estimator.inputs.numpy_input_fn)

Help on function numpy_input_fn in module tensorflow.python.estimator.inputs.numpy_io:

numpy_input_fn(x, y=None, batch_size=128, num_epochs=1, shuffle=None, queue_capacity=1000, num_threads=1)
    Returns input function that would feed dict of numpy arrays into the model.
    
    This returns a function outputting `features` and `targets` based on the dict
    of numpy arrays. The dict `features` has the same keys as the `x`. The dict
    `targets` has the same keys as the `y` if `y` is a dict.
    
    Example:
    
    ```python
    age = np.arange(4) * 1.0
    height = np.arange(32, 36)
    x = {'age': age, 'height': height}
    y = np.arange(-32, -28)
    
    with tf.Session() as session:
      input_fn = numpy_io.numpy_input_fn(
          x, y, batch_size=2, shuffle=False, num_epochs=1)
    ```
    
    Args:
      x: numpy array object or dict of numpy array objects. If an array,
        the array will be treated as a single feature.
      y: numpy array object or dict of nump

### Prepare input function

In [10]:
# Make input function for training: 
#   num_epochs=None -> will cycle through input data forever
#   shuffle=True -> randomize order of input data
# training_input_fn = tf.estimator.inputs.pandas_input_fn(
training_input_fn = tf.estimator.inputs.numpy_input_fn(
    x=X, 
    y=y, 
    batch_size=128,  # 拿掉 batch size accuracy 可由 0.93 提高到 0.96! 執行該 function 可看到 default 就是 128 
    shuffle=True, 
    num_epochs=None,  # Repeat forever
    )

### Define classifier 所需要的 feature_columns


In [5]:
%run feature_columns_for_DNN.ipynb

<IPython.core.display.Javascript object>

Stored 'feature_columns' (list)


### 祭出 DNNClassifier 

Classifier 無法跨 notebook save-restore 只能當地定義，所以恐怕會有多處定義不一致的狀況發生。
可以引用同一 source code string 的方式解決。

In [ ]:
%%js
alert("DNNClassifier_Training.ipynb: 清掉整個 model 的動作，若有需要手動做。")
// 若想清除現有的神經網路，從頭重新訓練（而非疊加上去），將 project 根目錄下的 model 子目錄整個刪掉，重新訓練。

In [11]:

classifier = tf.estimator.DNNClassifier(
    hidden_units=[24, 24, 24],  # 只隨便用了小小三層神經網路兒，對付 WH300 已經很好了 
    # feature_columns=feature_columns, 
    n_classes=2, 
    model_dir='model/dnn')

TypeError: __init__() missing 1 required positional argument: 'feature_columns'

### 開始訓練

In [7]:
classifier.train(input_fn=training_input_fn, steps=2000)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from model/dnn\model.ckpt-2000
INFO:tensorflow:Saving checkpoints for 2001 into model/dnn\model.ckpt.
INFO:tensorflow:loss = 0.0029042198, step = 2001
INFO:tensorflow:global_step/sec: 298.118
INFO:tensorflow:loss = 0.010137888, step = 2101 (0.338 sec)
INFO:tensorflow:global_step/sec: 381.495
INFO:tensorflow:loss = 0.0063057547, step = 2201 (0.261 sec)
INFO:tensorflow:global_step/sec: 385.67
INFO:tensorflow:loss = 0.0040197806, step = 2301 (0.259 sec)
INFO:tensorflow:global_step/sec: 391.073
INFO:tensorflow:loss = 0.0076776473, step = 2401 (0.256 sec)
INFO:tensorflow:global_step/sec: 372.118
INFO:tensorflow:loss = 0.0035029277, step = 2501 (0.270 sec)
INFO:tensorflow:global_step/sec: 437.401
INFO:tensorflow:loss = 0.013013186, step = 2601 (0.230 sec)
INFO:tensorflow:global_step/sec: 439.334
INFO:tensorflow:loss = 0.003639584, step = 2701 (0.227 sec)
INFO:tensorflow:global_step/sec: 452.048
INFO:tensorflow:l

# 完成！

此後 model_dir (即 model/dnn) 裡訓練好的 DNNClassifier 可以隨時調用了。

### 用 TensorBoard 查看訓練記錄圖表